# Importing required libraries

In [359]:
from dotenv import load_dotenv,find_dotenv

In [360]:
import pandas as pd

In [361]:
from langchain.chat_models import ChatOpenAI

In [362]:
from langchain.agents import create_pandas_dataframe_agent

In [363]:
import json

In [425]:
import openai
openai.api_type = "azure"
openai.api_key = "" # replace with your key
openai.api_base = "" # replace with your api_base
openai.api_version = "2023-03-15-preview"
deployment_id = 'gpt-4'

# Creating system message for OpenAI Chat Completion API

This will guarantee OpenAI understands the meaning of each column the dataframe I will read later

In [384]:
system_message = """Your name is Pepe and you are a data anaytics engineer. Your domain knowledge is about python, Pandas, and analytics. If you are asked about anything else, you reply "N/A". If you do not have enought information, reply "N/A" 
Description of some numeric columns the data frame:
"Unit price" is the price per unit 
"Quantity" is the number of items purchased 
"cogs" is the product of "unit price" and "Quantity"
"Tax 5%" is the tax calculated over the "cogs"
"Total" is the sum of "cogs" plus "Tax 5%"
"Date" is already in the Timestamp format 'YYYMMDD hhmmss'
"""

In [407]:
system_message = """Your name is Pepe and you are a data anaytics engineer. Your domain knowledge is about python, Pandas, and analytics. If you are asked about anything else, you reply "N/A". If you do not have enought information, reply "N/A" 

"""

In [408]:
system_message

'Your name is Pepe and you are a data anaytics engineer. Your domain knowledge is about python, Pandas, and analytics. If you are asked about anything else, you reply "N/A". If you do not have enought information, reply "N/A" \n\n'

# Creating an instance of the Langchain chat

This command will pass its parameters to Azure OpenAI gtp-4 model

In [409]:
chat=ChatOpenAI(temperature=0.0,engine="gpt-4")
#chat=ChatOpenAI(model_kwargs={"engine":"gpt-4","messages":{"role": "system","content": system_message}},temperature=0.0)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


# Reading the data from csv

In [410]:
df=pd.read_csv("supermarket_sales - Sheet1.csv")

In [411]:
df

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,1/29/2019,13:46,Ewallet,40.35,4.761905,2.0175,6.2
996,303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,3/2/2019,17:16,Ewallet,973.80,4.761905,48.6900,4.4
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2/9/2019,13:22,Cash,31.84,4.761905,1.5920,7.7
998,347-56-2442,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.2910,69.1110,2/22/2019,15:33,Cash,65.82,4.761905,3.2910,4.1


In [412]:

# convert the 'Date' column to datetime format
df['Date']= pd.to_datetime(df['Date'])

# convert the 'Time' column to datetime format
df['Time']= pd.to_datetime(df['Time']).dt.time

C:\Users\jose.brunfman\AppData\Local\Temp\ipykernel_24092\4103110667.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time']= pd.to_datetime(df['Time']).dt.time


In [413]:
df['Date'][0]

Timestamp('2019-01-05 00:00:00')

# Creating an instance of the agent that will answer questions regarding our data frame as data analytics expert

In [414]:
agent=create_pandas_dataframe_agent(chat,df,verbose=True,engine="gpt-4",model_kwargs={"engine":"gpt-4","messages":{"role": "system","content": system_message}},temperature=0.0)

# Ready to start asking analytics questions in natural language

As you can see, lanchin is verbose, explaining all the steps required to arrive to the correct answer 

In [419]:
question=input("Enter your question here: ")

Enter your question here: show what you understand each column means


In [420]:
#prompt=system_message+"\n"+question

In [424]:
try:
    answer=agent.run(question)
    
except Exception as e:
    # Print the error message
    print("An error occurred:", str(e))
    answer="N/A"
    



> Entering new AgentExecutor chain...
An error occurred: Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.


In [223]:
print(answer)

The totals of the 'Total' column separated by month are:
- January: 116,291.87
- February: 97,219.37
- March: 109,455.51


In [383]:
system_message

'Your name is Pepe and you are a data anaytics engineer. Your domain knowledge is about python, Pandas, and analytics. If you are asked about anything else, you reply "N/A". If you do not have enought information, reply "N/A" \n\n'

In [203]:
df.groupby()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,20:33:00,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,10:37:00,Ewallet,604.17,4.761905,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,2019-01-29,13:46:00,Ewallet,40.35,4.761905,2.0175,6.2
996,303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,2019-03-02,17:16:00,Ewallet,973.80,4.761905,48.6900,4.4
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2019-02-09,13:22:00,Cash,31.84,4.761905,1.5920,7.7
998,347-56-2442,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.2910,69.1110,2019-02-22,15:33:00,Cash,65.82,4.761905,3.2910,4.1


In [210]:
df.groupby(df['Date'].dt.month)['Total'].count()

Date
1    352
2    303
3    345
Name: Total, dtype: int64

Another example

In [174]:
question="show total sales by city and gender"

In [175]:
#prompt=system_message+"\n"+question

In [176]:
try:
    answer=agent.run(question)
except Exception as e:
    # Print the error message
    print("An error occurred:", str(e))
    answer="N/A"



> Entering new AgentExecutor chain...
Thought: To show total sales by city and gender, I need to group the dataframe by 'City' and 'Gender' columns and then sum the 'Total' column for each group.
Action: python_repl_ast
Action Input: df.groupby(['City', 'Gender'])['Total'].sum()
Observation: City       Gender
Mandalay   Female    52928.2950
           Male      53269.3770
Naypyitaw  Female    61685.4630
           Male      48883.2435
Yangon     Female    53269.1670
           Male      52931.2035
Name: Total, dtype: float64
Thought:I now know the total sales by city and gender.
Final Answer: 
Mandalay:
- Female: 52928.2950
- Male: 53269.3770

Naypyitaw:
- Female: 61685.4630
- Male: 48883.2435

Yangon:
- Female: 53269.1670
- Male: 52931.2035

> Finished chain.


In [177]:
print(answer)

Mandalay:
- Female: 52928.2950
- Male: 53269.3770

Naypyitaw:
- Female: 61685.4630
- Male: 48883.2435

Yangon:
- Female: 53269.1670
- Male: 52931.2035


# In next example you ask result in a Dictionary format, that can be used later on in your application

In [426]:
question="create a dictionary with the total sales per city, not including taxes, where the keys are the city name and the value is the total sales of the city"

In [427]:
#prompt=system_message+"\n"+question

In [428]:
try:
    result=agent.run(question)
    
except Exception as e:
    # Print the error message
    print("An error occurred:", str(e))
    answer="N/A"



> Entering new AgentExecutor chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


Thought: To create a dictionary with the total sales per city, I need to group the dataframe by the 'City' column and then sum the 'cogs' column for each city. Finally, I will convert the result to a dictionary.
Action: python_repl_ast
Action Input: df.groupby('City')['cogs'].sum().to_dict()
Observation: {'Mandalay': 101140.64, 'Naypyitaw': 105303.53, 'Yangon': 101143.21}
Thought:I now know the final answer
Final Answer: {'Mandalay': 101140.64, 'Naypyitaw': 105303.53, 'Yangon': 101143.21}

> Finished chain.


### Transforming result in a Python Dictionary

In [429]:
result=result.replace("'", "\"")

In [430]:
result=json.loads(result)

In [431]:
type(result)

dict

In [432]:
for city in result:
    print(city,result[city])

Mandalay 101140.64
Naypyitaw 105303.53
Yangon 101143.21


### As you can see above, answer is a dictionary that can be further used in the application